#### Analytical Function Estimator: Reinforcement Learning


In [7]:
from packages.func import func
import numpy as np
import random
from math import cos
import matplotlib.pyplot as plt
import sympy

ModuleNotFoundError: No module named 'sympy'

In [ ]:

def get_eval_points():
    num_samples = 100
    start = -15
    end = 15
    step = (end - start)/num_samples
    return np.arange(start, end, step)

def generate_sample(f: func) -> (np.array, np.array):
  
    eval_points = get_eval_points()
    # Modeling noise
    mean = 0
    std = np.random.uniform(0.1, 0.3, 1)
    noise = np.random.normal(mean, std, size=len(eval_points))

    X = f.eval(eval_points)
    X = X + noise

    return np.array(X)

In [ ]:
# State Representation
def initial_state():
    return func("0")  # Empty tree to start with

"""
Action Space: All possible operations on the tree that should be made by the
agent. Make sure to implement them in the class. 
"""

# Environment Simulation
def step(state, action):
    # Apply action to the state
    if action == "add_const":
        new_state = state + func("1")
    elif action == "add_x":
        new_state = state + func("x")
    elif action == "del_x":
        new_state = state - func("x")
    elif action == "del_const":
        new_state = state - func("1")

    return new_state

In [ ]:
# TODO: include the fourier analysis as a reward

def reward(state: func, data: np.array):
    
    # Evaluate the current state and return a reward
    try:
        eval_state = state.eval(get_eval_points())
        diff = eval_state - data
        squared_sum = 0
        for d in diff:
            squared_sum += np.square(d)
        error = np.sqrt(squared_sum)
        print("Error ", error)
        #complexity_penalty = state.get_size() # Simple complexity measure
        #print("Compl Penalty ", complexity_penalty)
        
        return -0.1*error
    
    except:
        print("We had a problem fellas")
        return -float('inf')  # Penalize invalid states


In [ ]:
f = func("x + 3")
data = generate_sample(f)
state = initial_state()

actions = ["add_const", "add_x", "del_x", "del_const"]

In [ ]:

# Q-Learning
Q = {}
learning_rate = 0.1
discount_factor = 0.5
epsilon = 0.2  # Exploration rate

f = func("2*x + 3")
data = generate_sample(f)

def q_learning_episode():
    max_iter = 0
    state = initial_state()
    while max_iter < 300:
        if random.uniform(0, 1) < epsilon:
            action = random.choice(actions)  # Explore
        else:
            q_values = [Q.get((state, a), 0) for a in actions]
            max_q = max(q_values)
            action = actions[q_values.index(max_q)]  # Exploit

        new_state = step(state, action)
        reward_value = reward(new_state, data)
        print(f"We chose {action} with rewarded value {reward_value} and size {new_state.get_size()}")
      
        if reward_value > -5:
            print("Good accuracy achieved with fct ", new_state.get_value())
            X_est = new_state.eval(get_eval_points())
            #print(get_eval_points())
            plt.plot(get_eval_points(), X_est)
            plt.plot(get_eval_points(), data)
            plt.show()
            return True

        # Update Q-value
        old_q = Q.get((state, action), 0)
        max_future_q = max([Q.get((new_state, a), 0) for a in actions])
        new_q = (1 - learning_rate) * old_q + learning_rate * (reward_value + discount_factor * max_future_q)
        Q[(state, action)] = new_q

        state = new_state

        max_iter += 1
        print("\n\n")
       

# Run episodes
for _ in range(100):
    if(q_learning_episode()):
        break


Error  174.12289784394767
We chose add_const with rewarded value -17.41228978439477 and size 2



Error  173.41418914137097
We chose add_const with rewarded value -17.341418914137098 and size 3



Error  173.28063491788262
We chose add_const with rewarded value -17.328063491788264 and size 4



Error  173.723561670632
We chose add_const with rewarded value -17.3723561670632 and size 5



Error  174.73858566875037
We chose add_const with rewarded value -17.473858566875037 and size 6



Error  176.31582675273535
We chose add_const with rewarded value -17.631582675273535 and size 7



Error  262.0936715562815
We chose del_x with rewarded value -26.20936715562815 and size 8



Error  263.5845027914906
We chose add_const with rewarded value -26.35845027914906 and size 9



Error  265.44394427792594
We chose add_const with rewarded value -26.544394427792597 and size 10



Error  267.6643140125481
We chose add_const with rewarded value -26.766431401254813 and size 11



Error  270.23671556210

We chose add_const with rewarded value -30.637985326580832 and size 33



Error  314.678108436484
We chose add_const with rewarded value -31.467810843648405 and size 34



Error  275.1899162811248
We chose add_x with rewarded value -27.51899162811248 and size 35



Error  259.69148641505643
We chose add_x with rewarded value -25.969148641505644 and size 36



Error  275.1899162811248
We chose del_x with rewarded value -27.51899162811248 and size 37



Error  284.69841493200806
We chose add_const with rewarded value -28.469841493200807 and size 38



Error  294.23950262801765
We chose add_const with rewarded value -29.423950262801768 and size 39



Error  303.8101090299146
We chose add_const with rewarded value -30.381010902991463 and size 40



Error  313.4075298884089
We chose add_const with rewarded value -31.34075298884089 and size 41



Error  323.0293751855073
We chose add_const with rewarded value -32.302937518550735 and size 42



Error  309.6904185253206
We chose add_x with rew

KeyboardInterrupt: 